In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

Every word will receive a code, so if we see the phrase I love my cat, we can use the previous words and assign a token to Cat, which is the one we don't know. Tensorflow and Keras gives us some good APIs to deal with this.

Tensorflow and keras give us a lot of ways to encode words, but we will focus on the tokenizer. The tokenizer generates the dictionary of word encoding and creating vectors from the sentences. If you are using a big dataset yo usually don't know how many words there are in the dataset, so we use a big number of words (`num_words`).  Turns your text into streams of tokens etc.

Tokenizer strips punctuation and is not case sensitive.

The reason why you would need this is that when it comes to training neural networks, you're going to be doing a lot of math and math deals with numbers, and instead of having the words being trained in a neural network, you can actually have the number representing that word and it just makes your life a lot easier.

`num_words` will take the 100 most common words or whatever value that you actually put in here. **The value is the token for the word.**

Now we are going to build the sentences into lists of tokens. The lists will have the same length, otherwise would be very hard to train the a neural network. It's the same process we followed when training convolutional neural networks, when the images don't fit the required size we had to rescale them, well, now we will do something similar.

In [4]:
sentences = ['I love my dog',
             'I love my cat',
             'I love you!',
             'Do you think my dog is amazing?']

# Declare the tokenizer instance
tokenizer = Tokenizer(num_words=100)
# takes in the data and ecodes it
# tokenizer provides a word_index property that returns a dictionary
# were the key is the word and the token is the value for that word

tokenizer.fit_on_texts(sentences)
word_index= tokenizer.word_index

# This will give me the set of sequences 
sequences = tokenizer.texts_to_sequences(sentences)

print(word_index)
print(sequences)

{'i': 1, 'love': 2, 'my': 3, 'dog': 4, 'you': 5, 'cat': 6, 'do': 7, 'think': 8, 'is': 9, 'amazing': 10}
[[1, 2, 3, 4], [1, 2, 3, 6], [1, 2, 5], [7, 5, 8, 3, 4, 9, 10]]


So the output is the dictionary and the list of sentences with the token replacing the words. The interesting thing is that the `text_to_sequences` can take any sentences and encode them based on the words that learned in `fit_on_text`.

In [6]:
test_data = ['i dont like your cat',
            'my dog loves to climb']

test_seq = tokenizer.texts_to_sequences(test_data)

print(word_index)
print(test_seq)

{'i': 1, 'love': 2, 'my': 3, 'dog': 4, 'you': 5, 'cat': 6, 'do': 7, 'think': 8, 'is': 9, 'amazing': 10}
[[1, 6], [3, 4]]


The resut is wrong because the model didn't learned some words from the test dataset. This means we need a lot of words to create a wide vocabularity. Also we can add some value to words, and add more value to unseen words.

We can use the `OOV` propierty to the tokenizer for out of vocabulary to use with words out of the vocabulay we fit.

In [8]:
tokenizer = Tokenizer(num_words=100, oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(sentences)

test_seq = tokenizer.texts_to_sequences(test_data)
print(word_index)
print(test_seq)

{'<OOV>': 1, 'i': 2, 'love': 3, 'my': 4, 'dog': 5, 'you': 6, 'cat': 7, 'do': 8, 'think': 9, 'is': 10, 'amazing': 11}
[[2, 1, 1, 1, 7], [4, 5, 1, 1, 1]]


Before training with text we need all sentences to have the same lenght, so we use padding. Tensorflow has an API for this.

In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [11]:
sentences = ['I love my dog',
             'I love my cat',
             'I love you!',
             'Do you think my dog is amazing?']

tokenizer = Tokenizer(num_words=100, oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences)

print(word_index, '\n')
print(sequences,'\n')
print(padded, '\n')

{'<OOV>': 1, 'i': 2, 'love': 3, 'my': 4, 'dog': 5, 'you': 6, 'cat': 7, 'do': 8, 'think': 9, 'is': 10, 'amazing': 11} 

[[2, 3, 4, 5], [2, 3, 4, 7], [2, 3, 6], [8, 6, 9, 4, 5, 10, 11]] 

[[ 0  0  0  2  3  4  5]
 [ 0  0  0  2  3  4  7]
 [ 0  0  0  0  2  3  6]
 [ 8  6  9  4  5 10 11]] 



We can see in the original sequences we had different lenghts and with padding they all have the same lenght now. Padding took the length of longer tokenized word and made the rest the same long by adding zeros. The padding can be added to after the sentece by adding `padding='post'`, you can also add a maximung lenght parameter `maxlen`. If you want the senteces to have a maximum length of 5 words, you can say `maxlen=5`. But, this can lead to loosing information from the begginig or the end of the word. By default we will loose from the beggining.

In [14]:
padded = pad_sequences(sequences, padding='post')

print(word_index, '\n')
print(sequences,'\n')
print(padded, '\n')

padded = pad_sequences(sequences, padding='post', maxlen=5)

print(word_index, '\n')
print(sequences,'\n')
print(padded, '\n')

{'<OOV>': 1, 'i': 2, 'love': 3, 'my': 4, 'dog': 5, 'you': 6, 'cat': 7, 'do': 8, 'think': 9, 'is': 10, 'amazing': 11} 

[[2, 3, 4, 5], [2, 3, 4, 7], [2, 3, 6], [8, 6, 9, 4, 5, 10, 11]] 

[[ 2  3  4  5  0  0  0]
 [ 2  3  4  7  0  0  0]
 [ 2  3  6  0  0  0  0]
 [ 8  6  9  4  5 10 11]] 

{'<OOV>': 1, 'i': 2, 'love': 3, 'my': 4, 'dog': 5, 'you': 6, 'cat': 7, 'do': 8, 'think': 9, 'is': 10, 'amazing': 11} 

[[2, 3, 4, 5], [2, 3, 4, 7], [2, 3, 6], [8, 6, 9, 4, 5, 10, 11]] 

[[ 2  3  4  5  0]
 [ 2  3  4  7  0]
 [ 2  3  6  0  0]
 [ 9  4  5 10 11]] 

